In [ ]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    

    

traindata = torchvision.datasets.CIFAR10(root='./cifar10', train=True,download=True, transform=transform)
                                        
trainloader = torch.utils.data.DataLoader(traindata, batch_size=4,shuffle=True, num_workers=2)
                                          

testdata = torchvision.datasets.CIFAR10(root='./cifar10', train=False,download=True, transform=transform)
                                       
testloader = torch.utils.data.DataLoader(testdata, batch_size=4,shuffle=False, num_workers=2)          

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 200)
        self.fc2 = nn.Linear(200, 10)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x




In [ ]:
# dev=torch.device('cuda')
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0005)

In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl
import torch_xla
import torch_xla.core.xla_model as xm

dev = xm.xla_device()

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        model=model.to(dev)
        # images, labels = data
        images, labels = data[0].to(dev), data[1].to(dev)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0



[1,   100] loss: 2.304
[1,   200] loss: 2.302
[1,   300] loss: 2.304
[1,   400] loss: 2.303
[1,   500] loss: 2.304
[1,   600] loss: 2.304
[1,   700] loss: 2.305
[1,   800] loss: 2.302
[1,   900] loss: 2.309


KeyboardInterrupt: ignored

In [ ]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
model = torch.load(PATH)
model.eval()